In [1]:
import os
import itertools

import numpy as np
import wandb
import seaborn as sn

import utils
from naive_model import NaiveModel
import encoding

NUM_EMBEDDING = 2000

In [4]:
import torch

import torch.nn as nn
assert torch.cuda.is_available()

def to_device(d):
    if hasattr(d, 'cuda'):
        return d.cuda()
    return {k: v.cuda() for k, v in d.items()}

class Model(nn.Module):
    def __init__(self, units, combinations=encoding.NAMES):
        super().__init__()
        self.units = units

        self.embed = nn.Embedding(num_embeddings=NUM_EMBEDDING, embedding_dim=units)
        self.lstm = nn.LSTM(input_size=units, hidden_size=units, num_layers=1, batch_first=True, bidirectional=True)

        self.tasks = {}
        for tup in combinations:
            shape = tuple(len(encoding.CLASSES[k]) for k in tup)
            out = nn.Linear(in_features=units, out_features=np.prod(shape))
            self.tasks[tup] = (out, shape)
            setattr(self, '+'.join(tup), out)

    def forward(self, x):
        embeds = self.embed(x)

        lstm_out, (h_n, c_n) = self.lstm(embeds)
        left, right = torch.chunk(h_n, 2, dim=0)
        merge = torch.squeeze(left + right)

        outputs = { k: f(merge).view(shape) for k, (f, shape) in self.tasks.items() }
        return outputs


In [11]:

def sanity():
    model = Model(100, combinations=[('B', 'T'), ('T', 'G')])
    with torch.no_grad():
        verbs = encoding.wordlist2numpy(["כשאתאקלם"])
        verbs = torch.from_numpy(verbs).to(torch.int64)
        tag_scores = model(verbs)
        for k in tag_scores:
            print(k)
            v = tag_scores[k].detach().numpy()
            print(v)
            print()

sanity()

('B', 'T')
[[-0.03073758  0.12819305 -0.07848827 -0.00512525]
 [ 0.03981335  0.08570858  0.04568484 -0.17644502]
 [ 0.22094595  0.15571463 -0.25609243  0.03385381]
 [-0.01417239  0.29071298  0.09653775 -0.07986734]
 [ 0.10159156  0.03317711 -0.06815157 -0.05779996]
 [ 0.11075711  0.02361325 -0.13230067  0.15099898]
 [-0.05639153  0.03840695  0.01949406 -0.05879   ]]

('T', 'G')
[[ 0.02805089 -0.10407012]
 [ 0.04608866  0.02199431]
 [-0.19873065 -0.13313344]
 [-0.05153071  0.13049652]]



In [ ]:
import concrete

def load_dataset(file_pat):
    *features_train, verbs_train = concrete.load_dataset(f'{file_pat}_train.tsv')
    *features_test, verbs_test = concrete.load_dataset(f'{file_pat}_test.tsv')
    return ((encoding.wordlist2numpy(verbs_train), encoding.list_of_lists_to_category(features_train)),
            (encoding.wordlist2numpy(verbs_test), encoding.list_of_lists_to_category(features_test)))

def load_dataset_split(filename, split):
    *features_train, verbs_train = concrete.load_dataset(filename)
    features_test = [t[-split:] for t in features_train]
    verbs_test = verbs_train[-split:]
    del verbs_train[-split:]
    for t in features_train:
        del t[-split:]
    return ((wordlist2numpy(verbs_train), list_of_lists_to_category(features_train)),
            (wordlist2numpy(verbs_test ), list_of_lists_to_category(features_test )))


In [ ]:
BATCH_SIZE = 128

def batch(a):
    ub = a.shape[0] // BATCH_SIZE * BATCH_SIZE
    return to_device(torch.from_numpy(a[:ub]).to(torch.int64)).split(BATCH_SIZE)

def batch_all_ys(ys):
    res = []
    m = {k: batch(ys[k]) for k in encoding.NAMES}
    nbatches = len(m['B'])
    for i in range(nbatches):
        res.append({k: m[k][i] for k in encoding.NAMES})
    return res

def fit(model, train, test, *, epochs,  runsize, criterion, optimizer, phases, teacher):
    x_train, y_train = train
    x_test, y_test = test
    data = {
        'train': (batch(x_train), batch_all_ys(y_train)),
        'test':  (batch(x_test ), batch_all_ys(y_test ))
    }

    stats = utils.Stats(model.tasks.keys(), runsize)
    
    for epoch in range(epochs):
        stats.epoch_start()
        
        for phase in phases:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            stats.phase_start(phase, batches_in_phase=len(data[phase][0]))

            for inputs, labels in zip(*data[phase]):
                stats.batch_start()
                
                if phase == 'train':
                    outputs = model(inputs)
                else:
                    with torch.no_grad():
                        outputs = model(inputs)

                if teacher is not None:
                    pseudo_labels = teacher(inputs)
                    losses = {k: criterion(outputs[k].double(), pseudo_labels[k]) for k in outputs}
                else:
                    losses = {k: criterion(outputs[k].double(), labels[k]) for k in outputs}
                
                if phase == 'train' and isinstance(criterion, nn.CrossEntropyLoss):
                    stats.assert_resonable_initial(losses)
                
                loss = sum(losses.values())

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                
                stats.update(loss=loss.item(),
                             batch_size=inputs.size(0),
                             d={k: (outputs[k], labels[k].detach()) for k in outputs})
                
                stats.batch_end()
            stats.phase_end()
        stats.epoch_end()
    return stats

@torch.no_grad()
def predict(model, *verbs):
    model.eval()
    verbs = encoding.wordlist2numpy(verbs)
    verbs = to_device(torch.from_numpy(verbs).to(torch.int64))
    outputs = model(verbs)
    res = {k: encoding.from_category(k, torch.argmax(v))
           for k, v in outputs.items()}
    res['R'] = ''.join(res[k] for k in ['R1', 'R2', 'R3', 'R4']).replace('.', '')
    return res
    

In [ ]:
arity = '3'
gen = 'all'
artifact_name = f'{gen}_{arity}_shuffled'
filename = f'synthetic/{artifact_name}.tsv'  # all_verbs_shuffled
test_size = 5000

artifact = wandb.Artifact(artifact_name, type='dataset')
artifact.add_file(filename)

train, test = load_dataset_split(filename, split=test_size)

In [ ]:
def naive_config(filename):
    return {
        'model': NaiveModel.learn_from_file(filename),
        'phases': ['test'],
        'criterion': nn.CrossEntropyLoss(),
        'optimizer': None
    }

def teacher_config(train):
    res = standard_config()
    res['teacher'] = NaiveModel.learn_from_data(train)
    res['criterion'] = nn.BCEWithLogitsLoss()  # BCELoss: works, but total loss is nan
    return res


def standard_config(model, lr):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    return {
        'model': model,
        'criterion': nn.CrossEntropyLoss(),
        'optimizer': optimizer,
        'phases': ['train', 'test'],
        'teacher': None
    }

In [ ]:
%env WANDB_SILENT true
%env WANDB_MODE run
# os.environ['WANDB_MODE'] = 'run'  # 'dryrun'

config = {
    'optimizer': 'adam',
    'batch_size': BATCH_SIZE,
    'epochs': 1,
    'runsize': 2 * 8192 // BATCH_SIZE,
    'test_size': test_size,
}
# group = 

def experiment(names, root=True):
    units = 400
    lr = 8e-4

    config.update({
        'units': units,
        'lr': lr,
    })
    names_str = '+'.join(names)
    if root:
        names += ['R1', 'R2', 'R3', 'R4']
        isroot = ''
    else:
        isroot = 'no'
    run = wandb.init(project="rootem",
                     group=f'subsets-{gen}-{arity}-{isroot}root',  # f'lr_units_grid_search-{arity}-{wandb.util.generate_id()}',
                     name=f'{gen}-{arity}-{names_str}-{isroot}root',  # f'{gen}-{arity}-{lr:.0e}',# f'{arity}-batch_{BATCH_SIZE}', # f'all-{arity}-lr_{lr:.0e}-units_{units}',
                     tags=[gen, arity, 'synthetic', 'shuffle', 'no_prefix'],
                     config=config)

    run.use_artifact(artifact)

    wandb.config.update(config, allow_val_change=True)

    print(config)
    
    model = to_device(Model(units=units, names=names))
    if isinstance(model, nn.Module):
        wandb.watch(model)
        
    stats = fit(train=train,
        test=test,
        epochs=config['epochs'],
        runsize=config['runsize'],
        **standard_config(model, lr)
    )
    # wandb.save(f"simple_{arity}.h5")
    return stats

# lr = 8e-4, 10e-4, 20e-4, 30e-4, 40e-4, 50e-4, 60e-4]:
def nonempty_powerset(seq):
    return itertools.chain.from_iterable(itertools.combinations(seq, r) for r in range(1, len(seq)+1))

# for root in [False, True]:
#     for subnames in nonempty_powerset(encoding.NAMES[:-4]):
#         print(subnames)
stats = experiment(names=[], root=True)

In [ ]:
k = 'R1'
labels = [x[::-1] for x in CLASSES[k]]
ax = sn.heatmap(stats.confusion[k], xticklabels=labels, yticklabels=labels, square=True, robust=True, cmap="cividis")

In [ ]:
ax = sn.heatmap(stats.confusion_logprobs[k], xticklabels=labels, yticklabels=labels, square=True, robust=True, cmap="cividis")

In [ ]:
print(predict(model, 'סבסו'))
print(predict(model, 'מקדו'))
print(predict(model, 'נמזר'))
print(predict(model, 'כרדו'))

In [ ]:
print(predict(model, 'הבריל'))
print(predict(model, 'חגוו'))
print(predict(model, 'עגו'))
print(predict(model, 'צירלל'))

In [ ]:
print(predict(model, "השטקרפתי"))

In [ ]:
print(predict(model, "ישסו"))

In [ ]:
import importlib
import encoding
import naive_model
import utils
encoding = importlib.reload(encoding)
naive_model = importlib.reload(naive_model)
utils = importlib.reload(utils)
wandb = importlib.reload(wandb)
Stats = utils.Stats
NaiveModel = naive_model.NaiveModel

In [ ]:
import sklearn
labels = [x[::-1] for x in CLASSES['B']]
# wandb.plots.HeatMap(labels, labels, stats.confusion['B'])
help(wandb.plots.heatmap)

In [ ]:
import itertools

In [ ]:
print(list(nonempty_powerset(NAMES[:-4])))

In [ ]:
def combine_classes(a, b):
    return ['.'.join(x) for x in itertools.product(a, b)]

combine_classes(CLASSES['B'], CLASSES['V'])